In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline

import pandas as pd
import numpy as np
import math
import matplotlib as mplstyle
import matplotlib.pyplot as plt
import locale
from locale import atof
from dateutil import parser
from datetime import datetime

plt.style.use('fivethirtyeight')

In [166]:
df_vouchersBySchool = pd.read_csv('school_choice_data/imported_data/awards_by_school_16.csv')
df_specialEdEnrollment = pd.read_csv('school_choice_data/gov_data/EnrollmentEnglishLearnerSpecEdSchls_ALL.csv')
df_missingSpecialEdEnrollment = pd.read_csv('school_choice_data/imported_data/updatedEnrollment.csv')
df_vouchersByOgSchoolDistrict = pd.read_csv('school_choice_data/imported_data/voucher-students-by-original-school-district.csv')
df_publicSpecialEdEnrollment = pd.read_csv('school_choice_data/gov_data/PublicEnrollmentEnglishLearnerSpecEd.csv')
df_schoolGrades2015 = pd.read_csv('school_choice_data/imported_data/final-2016-af-school-grades.csv')

In [140]:
# Functions

# normalize numbers
def normalizeNums(num):
    if (math.isnan(num)):
        return None
    else:
        num = int(math.floor(num))
        return num
    
# multiply by 100 to convert from percent
def convertPercent(num):
    return num * 100

# normalize percent from x%
def normalizePerc(num):
    try:
        float(num.strip('%'))
    except AttributeError:
        return num
    return float(num.strip('%'))
    
# get average award
def averageAward(awardCount, awardAmount):
    if math.isnan(awardAmount) or math.isnan(awardCount):
        return None
    return round((awardAmount/awardCount),2)

# Convert NaN to zeros
# the zeros do not always indicate that no students 
# were in the program. schools with less than ten 
# students do not have to report 
def convertInt(num):
    if math.isnan(num):
        return 0
    else:
        return int(num)
    
# calculate percent
def calcPerc(num, denom):
    if num != 0:
        percent = (float(num/denom))*100
        return round(percent,2)
    else:
        return None
    
# calculate percent change
def percentChange(oldVal, newVal):
    val1 = float(oldVal)
    val2 = float(newVal)
    change = (val2-val1)/val1
    return float(change)

# fill in missing values
def fillInfo(choiceCell, backupCell):
    if pd.isnull(choiceCell):
        return backupCell
    elif math.isnan(choiceCell):
        return backupCell
    else:
        return choiceCell
    
# replace with None
def fillNone(num):
    if num == 1:
        return None
    else:
        return num 

In [84]:
# VOUCHER ENROLLMENT DATA

df_vouchersBySchool

def replaceSymbols(id):
    if id == '*':
        return 9
    elif id == '-':
        return 0
    else:
        return id

df_vouchersBySchool2016_filtered = df_vouchersBySchool
df_vouchersBySchool2016_filtered['Voucher enrollment 2016'] = df_vouchersBySchool2016_filtered['2016-2017'].apply(replaceSymbols)
# df_vouchersBySchool2016_filtered

# Trim out data from 2011-2015.
df_vouchersBySchool2016_trimmed = df_vouchersBySchool.drop(['2011-2012','2012-2013','2013-2014','2014-2015','2015-2016','2016-2017'], axis=1)
df_vouchersBySchool2016_trimmed

# Filter out schools with no voucher enrollment in 2016
# In the data, 1 == "-", and 9 == "Less than 10 voucher students"
# We want to filter out the "-" ones because they're basically zero.
df_vouchersBySchool2016_trimmed = df_vouchersBySchool2016_trimmed[df_vouchersBySchool2016_trimmed['Voucher enrollment 2016'] > 0]
# df_vouchersBySchool2016_trimmed


In [156]:
# INDIANA SCHOOL ENROLLMENT (ALL)

# Filter out the public schools.
# Public school IDs are numbers-only. Private school IDs have letters + numbers.
def filterPrivate(id):
    if pd.isnull(id):
        return True
    else:
        try:
            float(id)
        except ValueError:
            return False
        return True

df_specialEdEnrollment_filtered = df_specialEdEnrollment
df_specialEdEnrollment_filtered['filter'] = df_specialEdEnrollment_filtered['SCHL_NAME_CODE'].apply(filterPrivate)
df_specialEdEnrollment_filtered = df_specialEdEnrollment_filtered[df_specialEdEnrollment_filtered['filter'] == False]
df_specialEdEnrollment_filtered = df_specialEdEnrollment_filtered.reset_index()

# Tidy and rename columns
df_specialEdEnrollment_trimmed = df_specialEdEnrollment_filtered.drop(['index','CORP','CORP_NAME','ENR_ELL_N','ENR_ELL_RATIO', 'filter'], axis=1)
df_specialEdEnrollment_trimmed = df_specialEdEnrollment_trimmed.rename(columns = {'SCHL_NAME_CODE':'School No.','SCHL_NAME':'School Name','ENR_SPEC_ED_N':'Special ed enrollment 2016','ENR_SPEC_ED_RATIO':'Special ed ratio 2016','ENROLLMENT_N':'Total enrollment 2016'})
df_specialEdEnrollment_trimmed['Special ed ratio 2016'] = df_specialEdEnrollment_trimmed['Special ed ratio 2016'].apply(normalizePerc)
df_specialEdEnrollment_trimmed

# Tidy and rename updated enrollment columns
df_missingSpecialEdEnrollment = df_missingSpecialEdEnrollment.rename(columns = {'School ID':'School No.','SpecEd 2016':'Special ed enrollment 2016','SpecEd Perc':'Special ed ratio 2016','Enrollment 2016':'Total enrollment 2016'})
df_missingSpecialEdEnrollment['Special ed ratio 2016'] = df_missingSpecialEdEnrollment['Special ed ratio 2016'].apply(normalizePerc)
df_missingSpecialEdEnrollment

# Merge w/ other special education data
df_specialEdEnrollment_merged = df_specialEdEnrollment_trimmed.merge(df_missingSpecialEdEnrollment,on='School No.', how='outer')
df_specialEdEnrollment_merged['Special ed enrollment 2016'] = np.vectorize(fillInfo)(df_specialEdEnrollment_merged['Special ed enrollment 2016_x'],df_specialEdEnrollment_merged['Special ed enrollment 2016_y'])
df_specialEdEnrollment_merged['Special ed ratio 2016'] = np.vectorize(fillInfo)(df_specialEdEnrollment_merged['Special ed ratio 2016_x'],df_specialEdEnrollment_merged['Special ed ratio 2016_y'])
df_specialEdEnrollment_merged['Total enrollment 2016'] = np.vectorize(fillInfo)(df_specialEdEnrollment_merged['Total enrollment 2016_x'],df_specialEdEnrollment_merged['Total enrollment 2016_y'])
df_specialEdEnrollment_merged = df_specialEdEnrollment_merged.drop(['Special ed enrollment 2016_x','Special ed enrollment 2016_y','Special ed ratio 2016_x','Special ed ratio 2016_y','Total enrollment 2016_x','Total enrollment 2016_y'], axis=1)
df_specialEdEnrollment_merged
# df_specialEdEnrollment_merged.to_csv('school_choice_data/exported_data/df_specialEdEnrollment_merged.csv')

# Merge w/ voucher school data
df_vouchersSpecialEd_merged = df_vouchersBySchool2016_trimmed.merge(df_specialEdEnrollment_merged,on='School No.', how='outer')
df_vouchersSpecialEd_merged = df_vouchersSpecialEd_merged.drop(['School Name_y'], axis=1)
df_vouchersSpecialEd_merged = df_vouchersSpecialEd_merged.rename(columns = {'School Name_x':'School Name'})
# df_vouchersSpecialEd_merged


In [158]:
# SCHOOL GRADES

# Tidy and rename columns
df_schoolGrades2015_trimmed = df_schoolGrades2015.drop(['IDOE_CORPORATION_ID','CORPORATION_NAME'], axis=1)
df_schoolGrades2015_trimmed = df_schoolGrades2015_trimmed.rename(columns = {'IDOE_SCHOOL_ID':'School No.','SCHOOL_NAME':'School Name','2015 Grade':'2015_grade'})
# df_schoolGrades2015Trimmed

# Merge w/ voucher school data
df_vouchersSpecialEdGrades_merged = df_vouchersSpecialEd_merged.merge(df_schoolGrades2015Trimmed,on='School No.', how='outer')
df_vouchersSpecialEdGrades_merged = df_vouchersSpecialEdGrades_merged.drop(['School Name_y'], axis=1)
df_vouchersSpecialEdGrades_merged = df_vouchersSpecialEdGrades_merged.rename(columns = {'School Name_x':'School Name'})
# df_vouchersSpecialEdGrades_merged

# Filter out all schools without vouchers in 2016
df_vouchersSpecialEdGrades_merged = df_vouchersSpecialEdGrades_merged[df_vouchersSpecialEdGrades_merged['Voucher enrollment 2016'] > 0]
# df_vouchersSpecialEdGrades_merged

In [176]:
# Assign schools to parent school districts

# Tidy and rename
df_vouchersByOgSchoolDistrict_filtered = df_vouchersByOgSchoolDistrict.rename(columns = {'Corp. No.':'District No.','Corporation Name':'District Name'})
df_vouchersByOgSchoolDistrict_filtered = df_vouchersByOgSchoolDistrict_filtered.drop(['Voucher students from this district (2016-2017)'], axis=1)
# df_vouchersByOgSchoolDistrict_filtered

# Merge w/ school ID
df_vouchersSpecialEdGradesDistricts_merged = df_vouchersSpecialEdGrades_merged.merge(df_vouchersByOgSchoolDistrict_filtered,on='School No.', how='outer')
df_vouchersSpecialEdGradesDistricts_merged = df_vouchersSpecialEdGradesDistricts_merged.drop(['School Name_y'], axis=1)
df_vouchersSpecialEdGradesDistricts_merged = df_vouchersSpecialEdGradesDistricts_merged.rename(columns = {'School Name_x':'School Name'})
df_vouchersSpecialEdGradesDistricts_merged

df_vouchersSpecialEdGradesDistricts_merged.to_csv('school_choice_data/exported_data/df_vouchersSpecialEdGradesDistricts_merged.csv')

In [165]:
# Aggregate by parent school district
df_districtVoucherSpecialEdAggregate = df_vouchersSpecialEdGradesDistricts_merged.drop(['Special ed ratio 2016'], axis=1)
df_districtVoucherSpecialEdAggregate = df_districtVoucherSpecialEdAggregate.groupby('District No.').sum()
df_districtVoucherSpecialEdAggregate = df_districtVoucherSpecialEdAggregate.reset_index()
df_districtVoucherSpecialEdAggregate

# Recalculate special ed ratio
df_districtVoucherSpecialEdAggregate['Special ed ratio 2016'] = np.vectorize(calcPerc)(df_districtVoucherSpecialEdAggregate['Special ed enrollment 2016'],df_districtVoucherSpecialEdAggregate['Total enrollment 2016'])
df_districtVoucherSpecialEdAggregate

,District No.,Special ed enrollment 2016,Total enrollment 2016,Special ed ratio 2016
0,15,25.0,297.0,8.42
1,25,75.0,1069.0,7.02
2,35,114.0,1012.0,11.26
3,125,629.0,9483.0,6.63
4,225,677.0,10389.0,6.52
5,235,849.0,13046.0,6.51
6,255,824.0,12617.0,6.53
7,365,224.0,3058.0,7.33
8,370,60.0,1025.0,5.85
9,395,243.0,2362.0,10.29


In [175]:
# Merge in public school special ed enrollment data

# Tidy and clean columns
df_publicSpecialEdEnrollment_trimmed = df_publicSpecialEdEnrollment
df_publicSpecialEdEnrollment_trimmed = df_publicSpecialEdEnrollment_trimmed.drop(['ENR_ELL_N','ENR_ELL_RATIO'], axis=1)
df_publicSpecialEdEnrollment_trimmed = df_publicSpecialEdEnrollment_trimmed.rename(columns = {'CORP':'District No.','CORP_NAME':'District Name','ENR_SPEC_ED_N':'Public special ed enrollment 2016','ENR_SPEC_ED_RATIO':'Public special ed ratio 2016','ENROLLMENT_N':'Public total enrollment 2016'})
df_publicSpecialEdEnrollment_trimmed['Public special ed ratio 2016'] = df_publicSpecialEdEnrollment_trimmed['Public special ed ratio 2016'].apply(normalizePerc)
df_publicSpecialEdEnrollment_trimmed

# Merge
df_districtVoucherSpecialEdAggregate_merged = df_publicSpecialEdEnrollment_trimmed.merge(df_districtVoucherSpecialEdAggregate,on='District No.', how='outer')
df_districtVoucherSpecialEdAggregate_merged

df_districtVoucherSpecialEdAggregate_merged.to_csv('school_choice_data/exported_data/df_districtVoucherSpecialEdAggregate_merged.csv')